In [1]:
!pip install requests pytrends pandas -q

In [12]:
import os
os.environ['SERPAPI_KEY'] = 'f751a2517e851225c22dbc6142b10bc6b525020fc0d8a2ee02a7dcd9e4ba3633'
print('SERPAPI_KEY set (hidden)')

SERPAPI_KEY set (hidden)


In [13]:
import os, time, glob, json, re, random, requests
from typing import List, Optional
import pandas as pd
from pytrends.request import TrendReq

print("✓ Ready")

✓ Ready


## Config

In [14]:
KEYWORDS_FILE = 'keyword_mapping.csv'
START_DATE = '2004-01-01'
END_DATE = '2025-12-05'
ANCHOR = 'Rau má'
BATCH_SIZE = 4  # tổng 5 từ mỗi request (1 anchor + 4 địa điểm)
OUTPUT_DIR = 'dest_trends_raw'
DELAY = 0.5  # group delay seconds
DELAY_JITTER = 0
START_GROUP = 1
END_GROUP = None  # None = tất cả

USE_SERPAPI = True  # Bật SerpAPI
SERPAPI_KEY = os.getenv('SERPAPI_KEY')
if not SERPAPI_KEY:
    raise RuntimeError('Thiếu SERPAPI_KEY (export SERPAPI_KEY=...)')

os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Config ready: {BATCH_SIZE} extra + anchor, {START_DATE} to {END_DATE}, delay={DELAY}±{DELAY_JITTER}s, serpapi={USE_SERPAPI}")

Config ready: 4 extra + anchor, 2004-01-01 to 2025-12-05, delay=0.5±0s, serpapi=True


## Load Keywords

In [15]:
df = pd.read_csv(KEYWORDS_FILE)
keywords = df['normalized_name'].dropna().tolist()

batches = []
for i in range(0, len(keywords), BATCH_SIZE):
    batch = [ANCHOR] + keywords[i:i+BATCH_SIZE]
    batches.append(batch)

if END_GROUP:
    batches = batches[START_GROUP-1:END_GROUP]
else:
    batches = batches[START_GROUP-1:]

print(f"✓ {len(keywords)} keywords → {len(batches)} batches")

✓ 967 keywords → 242 batches


## Fetch Function

In [16]:
def fetch_trends_serpapi(keywords: List[str], start_date: str, end_date: str) -> Optional[pd.DataFrame]:
    """Fetch qua SerpAPI (Google Trends engine)."""
    kw_list = keywords[:5]  # SerpAPI/Trends tối đa 5 từ (kể cả anchor)
    if len(keywords) > 5:
        print("    Warn: chỉ lấy 5 keyword đầu do giới hạn Trends")

    params = {
        "engine": "google_trends",
        "q": ",".join(kw_list),  # SerpAPI expects comma-separated string for multiple kws
        "data_type": "TIMESERIES",
        "cat": "0",
        "geo": "VN",
        "hl": "vi",
        "date": f"{start_date} {end_date}",
        "api_key": SERPAPI_KEY,
    }

    try:
        resp = requests.get("https://serpapi.com/search", params=params, timeout=30)
        resp.raise_for_status()
        data = resp.json()
        timeline = data.get("interest_over_time", {}).get("timeline_data", [])
        if not timeline:
            print("    Không có dữ liệu trả về")
            return None

        rows = []
        for entry in timeline:
            date = entry.get("date")
            for val in entry.get("values", []):
                query = val.get("query") or val.get("name")
                value = val.get("extracted_value")
                if value is None:
                    values_arr = val.get("value") or []
                    value = values_arr[0] if values_arr else None
                if query and date is not None:
                    rows.append({"date": date, "query": query, "value": value})

        if not rows:
            print("    Không parse được dữ liệu")
            return None

        df = pd.DataFrame(rows)
        df = df.pivot_table(index="date", columns="query", values="value")
        df = df.sort_index()
        return df
    except requests.HTTPError as e:
        status = e.response.status_code if e.response else "?"
        print(f"    SerpAPI HTTP {status}: {str(e)[:80]}")
    except Exception as e:
        print(f"    SerpAPI error: {str(e)[:80]}")
    return None


def fetch_trends(keywords: List[str], start_date: str, end_date: str) -> Optional[pd.DataFrame]:
    return fetch_trends_serpapi(keywords, start_date, end_date)


print("✓ Fetch functions ready (SerpAPI)")

✓ Fetch functions ready (SerpAPI)


In [11]:
# Check batch content
print(f"Batch 1 ({len(batches[0])} items): {batches[0]}")
print(f"\nBatch 2 ({len(batches[1])} items): {batches[1]}")
print(f"\nBatch 3 ({len(batches[2])} items): {batches[2]}")
print(f"\nBatch 5 ({len(batches[4])} items): {batches[4]}")

Batch 1 (5 items): ['Rau má', 'căn cứ địa cách mạng Hải Chi', 'Chợ Trung tâm Ba Chẽ', 'Miếu Ông – Miếu Bà', 'Phố đi bộ Tiên Yên']

Batch 2 (5 items): ['Rau má', 'Văn hóa, Thể thao các dân tộc vùng Đông Bắc', 'Đền thờ Đức ông Hoàng Cần', 'Chợ Tiên Yên', 'Thác Pạc Sủi']

Batch 3 (5 items): ['Rau má', 'Cột mốc 1317 và cửa khẩu Hoành Mô', 'Chợ Đồng Văn', 'Chợ Trung tâm huyện Bình Liêu', 'Đình Lục Nà']

Batch 5 (5 items): ['Rau má', 'Bãi tắm Quảng Hồng', 'Khoáng nóng Địa chất', 'lịch sử - Văn hóa Đền Cửa Ông', 'Vũng Đục']


## Fetch All

In [17]:
success = 0
failed = []

print(f"\n{'='*50}")
print(f"🚀 Fetching {len(batches)} batches...")
print(f"{'='*50}\n")

try:
    for idx, batch in enumerate(batches, start=START_GROUP):
        print(f"[{idx}] {batch[1][:30]}...")
        
        df = fetch_trends(batch, START_DATE, END_DATE)
        
        if df is not None:
            out_file = f"{OUTPUT_DIR}/dest_group_{idx:03d}.csv"
            df.to_csv(out_file)
            print(f"  ✓ {len(df)} rows")
            success += 1
        else:
            failed.append(idx)
            print(f"  ✗ Failed")
        
        if idx < START_GROUP + len(batches) - 1:
            # Thêm jitter (random) để tránh pattern
            wait_time = DELAY + random.uniform(0, DELAY_JITTER)
            time.sleep(wait_time)

except KeyboardInterrupt:
    print("\nInterrupted")

finally:
    print(f"\n{'='*50}")
    print(f"✅ Success: {success}/{len(batches)}")
    if failed:
        print(f"⚠️  Failed: {failed}")
    print(f"{'='*50}")


🚀 Fetching 242 batches...

[1] căn cứ địa cách mạng Hải Chi...
  ✓ 264 rows
  ✓ 264 rows
[2] Văn hóa, Thể thao các dân tộc ...
[2] Văn hóa, Thể thao các dân tộc ...
    Không có dữ liệu trả về
  ✗ Failed
    Không có dữ liệu trả về
  ✗ Failed
[3] Cột mốc 1317 và cửa khẩu Hoành...
[3] Cột mốc 1317 và cửa khẩu Hoành...
  ✓ 264 rows
  ✓ 264 rows
[4] Thác Sông Moóc...
[4] Thác Sông Moóc...
  ✓ 264 rows
  ✓ 264 rows
[5] Bãi tắm Quảng Hồng...
[5] Bãi tắm Quảng Hồng...
  ✓ 264 rows
  ✓ 264 rows
[6] Quảng Ninh Gate...
[6] Quảng Ninh Gate...
  ✓ 264 rows
  ✓ 264 rows
[7] cách mạng khu mỏ than Mạo Khê...
[7] cách mạng khu mỏ than Mạo Khê...
  ✓ 264 rows
  ✓ 264 rows
[8] Đền Thái...
[8] Đền Thái...
  ✓ 264 rows
  ✓ 264 rows
[9] Thác Mơ Quảng Ninh...
[9] Thác Mơ Quảng Ninh...
  ✓ 264 rows
  ✓ 264 rows
[10] Đền thờ Vua Lê Thái Tổ...
[10] Đền thờ Vua Lê Thái Tổ...
    Không có dữ liệu trả về
  ✗ Failed
    Không có dữ liệu trả về
  ✗ Failed
[11] Nhà thờ họ Lê...
[11] Nhà thờ họ Lê...
  ✓ 264 rows
 

## Verify

In [18]:
files = sorted(glob.glob(f"{OUTPUT_DIR}/dest_group_*.csv"))
print(f"✓ {len(files)} files")
if files:
    df = pd.read_csv(files[0], index_col=0)
    print(f"\nShape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    print(f"\n{df.head()}")

✓ 231 files

Shape: (264, 5)
Columns: ['Chợ Trung tâm Ba Chẽ', 'Miếu Ông – Miếu Bà', 'Phố đi bộ Tiên Yên', 'Rau má', 'căn cứ địa cách mạng Hải Chi']

            Chợ Trung tâm Ba Chẽ  Miếu Ông – Miếu Bà  Phố đi bộ Tiên Yên  \
date                                                                       
thg 1 2004                   0.0                 0.0                 0.0   
thg 1 2005                   0.0                 0.0                 0.0   
thg 1 2006                   0.0                 0.0                 0.0   
thg 1 2007                   0.0                 0.0                 0.0   
thg 1 2008                   0.0                 0.0                 0.0   

            Rau má  căn cứ địa cách mạng Hải Chi  
date                                              
thg 1 2004     0.0                           0.0  
thg 1 2005     0.0                           0.0  
thg 1 2006     0.0                           0.0  
thg 1 2007     0.0                           0.0  
thg 1 2008 

In [19]:
# Ghi danh sách các group bị fail vào file
if 'failed' in locals() and failed:
    with open(f"{OUTPUT_DIR}/failed_groups.txt", "w") as f:
        f.write(f"Total batches: {len(batches)}\n")
        f.write(f"Success: {success}\n")
        f.write(f"Failed: {len(failed)}\n\n")
        f.write("Failed group IDs:\n")
        for group_id in failed:
            f.write(f"{group_id}\n")
    print(f"✓ Failed groups list saved to {OUTPUT_DIR}/failed_groups.txt")
    print(f"Failed groups: {failed}")
else:
    print("No failed groups")

✓ Failed groups list saved to dest_trends_raw/failed_groups.txt
Failed groups: [2, 10, 13, 56, 66, 70, 96, 112, 221, 241, 242]
